In [12]:
import sys
import os
#sys.path.append("/home/phi.richter/SOFTWARE_PHD/run_RRTMG_OOP")
sys.path.append("/home/philipp/Doktorandenzeit/SOFTWARE_PHD/run_RRTMG_OOP")
import rrtmg as FLUXES
import numpy as np

In [13]:
## Profiles
z = np.loadtxt("/home/philipp/Doktorandenzeit/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/z.csv", delimiter=",")#np.loadtxt("/home/phi.richter/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/z.csv", delimiter=",")
co2 = np.loadtxt("/home/philipp/Doktorandenzeit/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/co2.csv", delimiter=",")#np.loadtxt("/home/phi.richter/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/co2.csv", delimiter=",")
o3 = np.loadtxt("/home/philipp/Doktorandenzeit/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/o3.csv", delimiter=",")#np.loadtxt("/home/phi.richter/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/o3.csv", delimiter=",")
ch4 = np.loadtxt("/home/philipp/Doktorandenzeit/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/ch4.csv", delimiter=",")#np.loadtxt("/home/phi.richter/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/ch4.csv", delimiter=",")
n2o = np.loadtxt("/home/philipp/Doktorandenzeit/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/n2o.csv", delimiter=",")#np.loadtxt("/home/phi.richter/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/n2o.csv", delimiter=",")

OSError: /home/philipp/Doktorandenzeit/SOFTWARE_PHD/Total_Cloud_Water_retrieval/retrieval/trace_gases/z.csv not found.

In [3]:
import netCDF4 as nc
#with nc.Dataset("/mnt/beegfs/user/phi.richter/REMOVE/ERA5/ozone.nc", "r") as f:
with nc.Dataset("/home/philipp/Doktorandenzeit/DATA_PHD/ERA5/ozone.nc", "r") as f:
    o3 = f.variables['o3'][:]
    z_era5 = f.variables['z'][:]/9.80665*1e-3
    lat = np.argmin(np.abs(np.array(f.variables['latitude'][:])-81.95))
    lon = np.argmin(np.abs(np.array(f.variables['longitude'][:])-10.33))
print(lat, lon)

12 81


In [4]:
import scipy.interpolate
o3_f = scipy.interpolate.interp1d(np.array(z_era5[0,:,12,81]), np.array(o3[0,:,12,81]), fill_value="extrapolate")
o3 = o3_f(np.array(z))

In [5]:
o3

array([1.50331316e-08, 1.48546813e-08, 1.46847285e-08, 1.44722876e-08,
       1.42598467e-08, 1.40474059e-08, 1.38349650e-08, 1.36225241e-08,
       1.35800359e-08, 1.36330443e-08, 1.65466633e-08, 1.94602823e-08,
       2.23739013e-08, 3.69419963e-08, 5.15100913e-08, 5.73373293e-08,
       7.19054243e-08, 7.50226894e-08, 7.89712375e-08, 8.13403664e-08,
       8.17220283e-08, 8.17604282e-08, 8.18065082e-08, 8.25502069e-08,
       8.37538766e-08, 8.57322934e-08, 8.79473406e-08, 8.90635058e-08,
       8.89272266e-08, 8.79789800e-08, 8.69303568e-08, 8.68897517e-08,
       8.72658883e-08, 8.76497175e-08, 9.10108799e-08, 9.78669211e-08,
       1.02281495e-07, 1.06696070e-07, 1.10969413e-07, 1.12769660e-07,
       1.15470030e-07, 1.18689596e-07, 1.22418377e-07, 1.26397435e-07,
       1.31912645e-07, 1.37427855e-07, 1.43606820e-07, 1.50250504e-07,
       1.56894187e-07, 1.62106486e-07, 1.66974538e-07, 1.71842589e-07,
       1.95659367e-07, 2.23657104e-07, 2.73704593e-07, 3.96924905e-07,
      

In [6]:
path_retrievals = "/home/philipp/Doktorandenzeit/DATA_PHD/RRTMG/INPUT/INPUT_FOR_RRTMG/RRTMG_input_nomod"#"/mnt/beegfs/user/phi.richter/DATA_PHD/RRTMG/INPUT/INPUT_FOR_RRTMG/RRTMG_input_nomod"
ssp_ice = "./"
ssp_wat = "./"
files = sorted(os.listdir(path_retrievals))

In [7]:
path_rrtmg = "/home/philipp/Doktorandenzeit/SOFTWARE_PHD/run_RRTMG_OOP"#"/home/phi.richter/SOFTWARE_PHD/run_RRTMG_OOP"
model = FLUXES.RRTMG(path_rrtmg+"/rrtmg_lw_v5.00_linux_pgi", path_rrtmg+"/rrtmg_sw_v5.00_linux_pgi", \
                  [ssp_wat, ssp_ice])

In [25]:
epsilon = 0.99*np.ones(16)
epsilon[np.array([5,6,7])] = 0.98
spec = files[10000]
print(spec)
#atm_file = "/mnt/beegfs/user/phi.richter/ERA5_atm_interpolated/ERA5"+spec.split("in_CNET")[1].split(".nc")[0]+".csv"
model.read_cloudnet(os.path.join(path_retrievals, spec))#, pattern_fname="ERA5_%Y%m%d_%H%M%S.nc")
model.plot_atmosphere()
#model.replace_atmosphere(atm_file)
#model.plot_atmosphere()

in_CNET_20170606_204444_81.950648_10.539039_73.62477298934044.nc


In [26]:
o3_ppmv = 0.*np.ones(z.size)#np.ones(z.size)#28.9644 / 47.9982 * 1e6 * o3
co2 = np.zeros(z.size)
n2o = np.zeros(z.size)
ch4 = np.zeros(z.size)
tg = model.read_trace_gases(z, co2, n2o, ch4, o3_ppmv)
#model.plot_atmosphere()

In [38]:
epsilon = 0.99*np.ones(16)
atm = "H444444"#"HAAA4A4"
epsilon[np.array([7,8,9,10])] = 0.98
in_cld_rrtm = model.create_inputfile_cloud()
input_rrtm = model.create_inputfile_atm_terrestrial(cloud=0, semiss=epsilon, atm=atm)
model.run_RRTMG_terrestrial(clouds=False)
input_rrtm = model.create_inputfile_atm_terrestrial(cloud=2, semiss=epsilon, atm=atm)
model.run_RRTMG_terrestrial(clouds=True)
input_rrtm = model.create_inputfile_atm_solar(cloud=0, atm=atm, aerosols=10)
input_aer = model.create_inputfile_aerosols_solar(level=[1], aot=[0.05], num_aer=1, iaod=0, issa=0, ipha=0, aerpar=[0.13, 1.0, 0.0], ssa=[0.780], phase=[0.7])
model.run_RRTMG_solar(clouds=False)
input_rrtm = model.create_inputfile_atm_solar(cloud=2, atm=atm, aerosols=10)
model.run_RRTMG_solar(clouds=True)
model.write_results("/home/philipp/out1.nc")

In [39]:
model.get_fluxes_solar(30)['all']['DOWNWARD FLUX'].iloc[73]-model.get_fluxes_solar(30)['clear']['DOWNWARD FLUX'].iloc[73]

-36.529300000000006

In [11]:
print(input_aer)

    1
    1    0    0    0    0.13    1.00    0.00
    1 0.1900
 0.78
 0.70

